# 1. Read Data from folders

In [3]:
import cv2
import matplotlib.pyplot as plt
import os

class DirectoryBuddy:
    
    def __init__(self, parent_directory_path):
        
        if not os.path.isdir(parent_directory_path):
            raise ValueError("Invalid directory path.")
        
        self.parent_directory_path = parent_directory_path
        self.parent_directory = os.listdir(parent_directory_path)
    
    def rename_image_paths(self, show_path=False):
        
        for folder_name in self.parent_directory:
            folder_path = os.path.join(self.parent_directory_path, folder_name)
            
            for i, filename in enumerate(os.listdir(folder_path)):
                image_path = os.path.join(folder_path, filename)
                
                if not os.path.isfile(image_path):
                    continue
                
                new_image_name = os.path.join(folder_path, f"{i}.jpg")
                
                if show_path:
                    print(image_path+ " -> " + new_image_name)
                
                try:
                    os.rename(image_path, new_image_name)
                except OSError as e:
                    print(f"Error renaming file: {e}")
    
    def show_image(self, image_path):
        
        try:
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            plt.imshow(img)
            plt.axis('off')
            plt.show()
        except:
            pass
        
       
        
    def show_image(self, image):
        try:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            plt.imshow(image)
            plt.axis('off')
            plt.show()
        except:
            pass              
            
            
    def create_image_array(self,show=False):
        
        folder_dictionary={}
        for folder_name in self.parent_directory:
            folder_path = os.path.join(self.parent_directory_path, folder_name)
            
            image_array=[]
            
            
            for i, filename in enumerate(os.listdir(folder_path)):
                image_path = os.path.join(folder_path, filename)
                
                image=cv2.imread(image_path)
                image_array.append(image)  
                if show:
                    self.show_image(image)
                    
            folder_dictionary[folder_name]=[image_array,image_path]
            
        return folder_dictionary
    
    def export_to_directory(self, export_directory,callback_function):
    
        if not os.path.isdir(export_directory):
            os.mkdir(export_directory)
            
        for folder_name in self.parent_directory:
            folder_path=os.path.join(self.parent_directory_path,folder_name)#../raw/good
            folder=os.listdir(folder_path)
            folder.sort()
            
            export_path=os.path.join(export_directory,folder_name)
            print(export_path)
            
            try :
                os.mkdir(export_path)
            except:
                pass    
                    
            for i in range(0,len(folder)):
            
                image_path=os.path.join(folder_path,folder[i])
                export_image_path=os.path.join(export_path,folder[i])
                
                box=callback_function(image_path)
                flag=cv2.imwrite(export_image_path,box)
                
                print(export_image_path + ' : ' + str(flag))    
                
            

    

In [4]:
director_buddy = DirectoryBuddy('../raw')

# 2. YOLO object detection for humans

In [5]:
from super_gradients.training import models
import matplotlib.pyplot as plt
import cv2

class DetectBuddy():
    
    def __init__(self,model_name='yolo_nas_l',weights='coco',conf_threshold=0.8):
        self.model = models.get(model_name, pretrained_weights=weights)
        self.conf_threshold=conf_threshold
        self.detection_list=[]
        
    def predict(self,image_path,show=False):
        
        prediction = self.model.predict(image_path, conf=self.conf_threshold)
        if show:
            prediction.show()
        
        detection_pred = prediction._images_prediction_lst
        self.detection_list = list(detection_pred)
        
    
    
    def predict(self,image,show=False):
        prediction = self.model.predict(image, conf=self.conf_threshold)
        if show:
            prediction.show()
        
        detection_pred = prediction._images_prediction_lst
        self.detection_list = list(detection_pred)
        
    
    def extract_human_being(self,img,show=False):
        
        self.predict(img)
            
        if(len(self.detection_list)==0):
            return None
        
        bboxes_xyxy = self.detection_list[0].prediction.bboxes_xyxy.tolist()
        
        ymin = int(bboxes_xyxy[0][1])
        ymax = int(bboxes_xyxy[0][3])
        xmin = int(bboxes_xyxy[0][0])
        xmax = int(bboxes_xyxy[0][2])
        
        if type(img) == str:
            img = cv2.imread(img)

        
        box = img[ymin:ymax, xmin:xmax]
        if show:
            director_buddy.show_image(box)
    
        return box  

# 3. Use Image Classifier to classify the images